# Librerías

In [64]:
from ortools.sat.python import cp_model

# Modelo CSP

In [65]:
#Crear un modelo CSP
model = cp_model.CpModel()

# Datos

In [66]:
#Datos
Stewards =  ['Tom','David','Jeremy','Ron','Joe','Bill','Fred','Bob','Mario','Ed']
Hostesess = ['Carol','Janet','Tracy','Marilyn','Carolin','Katy','Ines','Jean','Heather','Juliet']
n_stewards  = len(Stewards)
n_hostesess = len(Hostesess)

Crew = Stewards + Hostesess
n_crew = len(Crew)
  
gender = [1 for _ in range(n_stewards)] + [0 for _ in range(n_hostesess)]

n_flights = 10
Aircrew = [4,5,5,6,7,4,5,6,6,7]

min_hostesess = [1,1,1,2,3,1,1,1,2,3]
min_stewards =  [1,1,1,2,3,1,1,1,2,3]

french  = [Crew.index('Ines'), Crew.index('Bill'), Crew.index('Jean'), Crew.index('Juliet')]
spanish = [Crew.index('Tom'), Crew.index('Jeremy'), Crew.index('Mario'), Crew.index('Katy'), Crew.index('Juliet')]
german  = [Crew.index('Bill'), Crew.index('Fred'), Crew.index('Joe'), Crew.index('Mario'), Crew.index('Marilyn'), Crew.index('Ines'), Crew.index('Heather')]

# Variables y Dominios

In [67]:
#Variables y dominios: X:un conjunto de variables, D: un conjunto de dominios

#G_reference = [[False for f in range(n_flights)] for c in range(n_crew)]
G = [[model.NewBoolVar(f"{Crew[c]}-{f}") for f in range(n_flights)] for c in range(n_crew)]
for c in G:
    print(c)

[Tom-0(0..1), Tom-1(0..1), Tom-2(0..1), Tom-3(0..1), Tom-4(0..1), Tom-5(0..1), Tom-6(0..1), Tom-7(0..1), Tom-8(0..1), Tom-9(0..1)]
[David-0(0..1), David-1(0..1), David-2(0..1), David-3(0..1), David-4(0..1), David-5(0..1), David-6(0..1), David-7(0..1), David-8(0..1), David-9(0..1)]
[Jeremy-0(0..1), Jeremy-1(0..1), Jeremy-2(0..1), Jeremy-3(0..1), Jeremy-4(0..1), Jeremy-5(0..1), Jeremy-6(0..1), Jeremy-7(0..1), Jeremy-8(0..1), Jeremy-9(0..1)]
[Ron-0(0..1), Ron-1(0..1), Ron-2(0..1), Ron-3(0..1), Ron-4(0..1), Ron-5(0..1), Ron-6(0..1), Ron-7(0..1), Ron-8(0..1), Ron-9(0..1)]
[Joe-0(0..1), Joe-1(0..1), Joe-2(0..1), Joe-3(0..1), Joe-4(0..1), Joe-5(0..1), Joe-6(0..1), Joe-7(0..1), Joe-8(0..1), Joe-9(0..1)]
[Bill-0(0..1), Bill-1(0..1), Bill-2(0..1), Bill-3(0..1), Bill-4(0..1), Bill-5(0..1), Bill-6(0..1), Bill-7(0..1), Bill-8(0..1), Bill-9(0..1)]
[Fred-0(0..1), Fred-1(0..1), Fred-2(0..1), Fred-3(0..1), Fred-4(0..1), Fred-5(0..1), Fred-6(0..1), Fred-7(0..1), Fred-8(0..1), Fred-9(0..1)]
[Bob-0(0..1),

# Simple Version

In [ ]:
#Restricciones: C:un conjunto de restricciones
#for f in range(n_flights):
#    #Crew Number
#    model.Add(sum([G[i][f] for i in range(n_crew)]) == Aircrew[f])
#    
#    #Crew Min
#    model.Add(sum([G[i][f] if(gender[i]==0) else 0 for i in range(n_crew)]) >= min_hostesess[f])
#    model.Add(sum([G[i][f] if(gender[i]==1) else 0 for i in range(n_crew)]) >= min_stewards[f])
#    
#    #Languages
#    model.Add(sum([G[i][f] if(i in french) else 0 for i in range(n_crew)]) >= 1)
#    model.Add(sum([G[i][f] if(i in spanish) else 0 for i in range(n_crew)]) >= 1)
#    model.Add(sum([G[i][f] if(i in german) else 0 for i in range(n_crew)]) >= 1)
#    
#    #Repetition
#    if(f>=2):
#        for c in range(n_crew):
#            model.Add(  G[c][f]+G[c][f-1]+G[c][f-2] <= 1 )

# Global Constraints and Reification Version

In [68]:
#Restricciones: C:un conjunto de restricciones

for f in range(n_flights):
    #Crew Number
    model.Add(sum([G[i][f] for i in range(n_crew)]) == Aircrew[f])
        
    #Crew Min
    model.Add(sum([G[i][f] if (gender[i] == 0) else 0 for i in range(n_crew)]) >= min_hostesess[f])
    model.Add(sum([G[i][f] if (gender[i] == 1) else 0 for i in range(n_crew)]) >= min_stewards[f])

    #Languages
    model.AddAtLeastOne([G[i][f] if (i in french) else 0 for i in range(n_crew)])
    model.AddAtLeastOne([G[i][f] if (i in spanish) else 0 for i in range(n_crew)])
    model.AddAtLeastOne([G[i][f] if (i in german) else 0 for i in range(n_crew)])

    #Repetition
    if f < n_flights - 2: 
        for i in range(n_crew):
            model.Add(  G[i][f+1]+G[i][f+2] == 0 ).OnlyEnforceIf(G[i][f])
    if f > 2: 
        for i in range(n_crew):
            model.Add(  G[i][f-1]+G[i][f-2] == 0 ).OnlyEnforceIf(G[i][f])

# Solver

In [69]:
#Crear un solver
solver = cp_model.CpSolver() 
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
    Header = [f'F{i+1}' for i in range(n_flights)]
    print("\t\t"+ "\t".join(Header))

    for j in range(n_crew):
        if j == 10: print()
        print(Crew[j],end='\t->\t')        
        for i in range(n_flights):
            print(solver.Value(G[j][i]),end='\t')      
        print() 

		F1	F2	F3	F4	F5	F6	F7	F8	F9	F10
Tom	->	0	0	1	0	0	1	0	0	1	0	
David	->	0	1	0	0	1	0	0	1	0	0	
Jeremy	->	1	0	0	1	0	0	1	0	0	1	
Ron	->	1	0	0	1	0	0	1	0	0	1	
Joe	->	0	0	1	0	0	1	0	0	1	0	
Bill	->	1	0	0	1	0	0	1	0	0	1	
Fred	->	0	0	1	0	0	0	1	0	0	1	
Bob	->	0	1	0	0	1	0	0	0	1	0	
Mario	->	0	1	0	0	1	0	0	1	0	0	
Ed	->	0	0	0	1	0	0	0	1	0	0	

Carol	->	0	0	0	0	1	0	0	0	0	1	
Janet	->	0	0	0	1	0	0	1	0	0	1	
Tracy	->	0	1	0	0	1	0	0	1	0	0	
Marilyn	->	1	0	0	1	0	0	0	1	0	0	
Carolin	->	0	0	0	0	1	0	0	0	1	0	
Katy	->	0	0	0	0	0	0	0	0	0	1	
Ines	->	0	0	1	0	0	1	0	0	1	0	
Jean	->	0	1	0	0	1	0	0	1	0	0	
Heather	->	0	0	1	0	0	1	0	0	1	0	
Juliet	->	0	0	0	0	0	0	0	0	0	0	
